# Dummy Tweet Scraping

### Import Libraries

In [13]:
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
import time
import datetime
import numpy as np
from cmath import nan
from html import entities

### Account Authentification

In [14]:
# Authenticating Twitter API
# Obtain your Twitter credentials from your twitter developer account

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEGchQEAAAAAZSYFv1nyLDV81YAKEfDr1fVrlho%3DWKBvyLhQ4CeHrlBRtecAetYkB1ZnAjI3Zydb1516fkIzKhS4vh'

client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

### Search Criteria
This is just an arbitary search criteria for Dummy results with the word protest in them

In [15]:
search_words = '(protest OR strike OR march OR rally OR #feesmustfall OR #zumamustfall) -is:retweet lang:en place_country:ZA'
#search_words = 'protest -is:retweet lang:en place_country:ZA'
date_since = datetime.datetime(2021,1,1)
date_before = datetime.datetime(2022,9,30)
numTweets = 500

Scrape Tweets

In [16]:
# Scrape Tweets
# Note for the final versions we should use -> until_id -> allos to resume scraping from a particualar tweet ID
protest_tweets = []
count = 0 
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = search_words,
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 place_fields = ['id', 'geo', 'name', 'country_code', 'place_type', 'full_name', 'country'],
                                 expansions = ['author_id', 'geo.place_id'],
                                 start_time = date_since,
                                 end_time = date_before,
                                 max_results=numTweets
                              ):
   time.sleep(1)
   protest_tweets.append(response)
   # count = count +1
   # if (count == 4):
   #    break

Populate Dataframe with tweet data

In [17]:
error_count = 0
result = []
user_dict = {}
place_dict = {}
# Loop through each response object
for response in protest_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location,
                             }
    
    place_dict = {p['id']: p for p in response.includes['places']}

    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet

        try:
            if place_dict[tweet.geo['place_id']]:
                place = place_dict[tweet.geo['place_id']]   
            else:
                place = nan

            result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count'],
                       'place_id': tweet.geo,
                       'place_name': place.full_name,
                       'bbox': place.geo['bbox']

                      })

        except:
            error_count = error_count + 1


print(error_count, " result(s) were excluded due to invalid data points.")        

# Change this list of dictionaries into a dataframe             
df = pd.DataFrame(result)                

5  result(s) were excluded due to invalid data points.


In [18]:
# (number of tweets, number of attributes)
df.shape

(21655, 15)

In [19]:
df.head()

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,1353804645580353537,ranthotse17,1433,2648,all is well,Limpopo,Go shota R1000 ya registration \n\nBlesser: +R...,2022-09-29 19:40:06+00:00,0,0,1,0,{'place_id': 'dd9c0d7d7e07eb49'},South Africa,"[16.4475932, -34.8342468, 32.8922934, -22.1247..."
1,1258457287120818182,Michael86259668,1323,29766,plus500 associate commodity fx and cfd trader ...,South Africa,Spx yesterday's rally\nPoof gone https://t.co/...,2022-09-29 14:17:35+00:00,0,0,1,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681..."
2,2496356957,KabeloMG,1436,35414,This is Twitter.We tweet like birds hence the ...,None,@JohnPerlman A driverless car wouldn't work in...,2022-09-29 13:49:14+00:00,1,0,0,0,{'place_id': '0e587c59401d0a27'},"Pretoria, South Africa","[27.9483035, -25.9157727, 28.4198285, -25.5894..."
3,1309123451433746437,TheLegitIcon2,6458,37979,"Phumlani, 𝒍𝒐𝒗𝒆𝒓, 𝒇𝒖𝒕𝒖𝒓𝒆 𝒍𝒆𝒂𝒅𝒆𝒓 𝒂𝒏𝒅 𝒂 𝒈𝒐𝒐𝒅 𝒄𝒉𝒂𝒓...","Pietermaritzburg, South Africa",At least offer ukumbhalela ama board for that ...,2022-09-29 11:10:06+00:00,0,0,1,0,{'place_id': '52e073e7724385c3'},"Pietermaritzburg, South Africa","[30.2563496, -29.6999989, 30.4512134, -29.5328..."
4,177685467,Nwabisa_1,1878,52381,"mamCirha, Qhanqolo, Ncibane , Nojawolo, Ntswen...",Joburg,We used to catch a train from Ikwezi to Mzimhl...,2022-09-29 10:52:13+00:00,1,0,1,0,{'place_id': '3e46a98adcf05e59'},"Meadowlands, South Africa","[27.8651453, -26.2359005, 27.9219768, -26.1964..."


Save data to csv

In [20]:
df.to_csv('SA_Tweets_All_Locations_v2.csv')

In [21]:
db = pd.read_csv('SA_Tweets_All_Locations_v2.csv')
db.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,0,1353804645580353537,ranthotse17,1433,2648,all is well,Limpopo,Go shota R1000 ya registration \n\nBlesser: +R...,2022-09-29 19:40:06+00:00,0,0,1,0,{'place_id': 'dd9c0d7d7e07eb49'},South Africa,"[16.4475932, -34.8342468, 32.8922934, -22.1247..."
1,1,1258457287120818182,Michael86259668,1323,29766,plus500 associate commodity fx and cfd trader ...,South Africa,Spx yesterday's rally\nPoof gone https://t.co/...,2022-09-29 14:17:35+00:00,0,0,1,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681..."
2,2,2496356957,KabeloMG,1436,35414,This is Twitter.We tweet like birds hence the ...,NaN,@JohnPerlman A driverless car wouldn't work in...,2022-09-29 13:49:14+00:00,1,0,0,0,{'place_id': '0e587c59401d0a27'},"Pretoria, South Africa","[27.9483035, -25.9157727, 28.4198285, -25.5894..."
3,3,1309123451433746437,TheLegitIcon2,6458,37979,"Phumlani, 𝒍𝒐𝒗𝒆𝒓, 𝒇𝒖𝒕𝒖𝒓𝒆 𝒍𝒆𝒂𝒅𝒆𝒓 𝒂𝒏𝒅 𝒂 𝒈𝒐𝒐𝒅 𝒄𝒉𝒂𝒓...","Pietermaritzburg, South Africa",At least offer ukumbhalela ama board for that ...,2022-09-29 11:10:06+00:00,0,0,1,0,{'place_id': '52e073e7724385c3'},"Pietermaritzburg, South Africa","[30.2563496, -29.6999989, 30.4512134, -29.5328..."
4,4,177685467,Nwabisa_1,1878,52381,"mamCirha, Qhanqolo, Ncibane , Nojawolo, Ntswen...",Joburg,We used to catch a train from Ikwezi to Mzimhl...,2022-09-29 10:52:13+00:00,1,0,1,0,{'place_id': '3e46a98adcf05e59'},"Meadowlands, South Africa","[27.8651453, -26.2359005, 27.9219768, -26.1964..."


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21655 entries, 0 to 21654
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   author_id           21655 non-null  int64              
 1   username            21655 non-null  object             
 2   author_followers    21655 non-null  int64              
 3   author_tweets       21655 non-null  int64              
 4   author_description  21655 non-null  object             
 5   author_location     19015 non-null  object             
 6   text                21655 non-null  object             
 7   created_at          21655 non-null  datetime64[ns, UTC]
 8   retweets            21655 non-null  int64              
 9   replies             21655 non-null  int64              
 10  likes               21655 non-null  int64              
 11  quote_count         21655 non-null  int64              
 12  place_id            21655 non-nu

In [23]:
df.head()

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,1353804645580353537,ranthotse17,1433,2648,all is well,Limpopo,Go shota R1000 ya registration \n\nBlesser: +R...,2022-09-29 19:40:06+00:00,0,0,1,0,{'place_id': 'dd9c0d7d7e07eb49'},South Africa,"[16.4475932, -34.8342468, 32.8922934, -22.1247..."
1,1258457287120818182,Michael86259668,1323,29766,plus500 associate commodity fx and cfd trader ...,South Africa,Spx yesterday's rally\nPoof gone https://t.co/...,2022-09-29 14:17:35+00:00,0,0,1,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681..."
2,2496356957,KabeloMG,1436,35414,This is Twitter.We tweet like birds hence the ...,None,@JohnPerlman A driverless car wouldn't work in...,2022-09-29 13:49:14+00:00,1,0,0,0,{'place_id': '0e587c59401d0a27'},"Pretoria, South Africa","[27.9483035, -25.9157727, 28.4198285, -25.5894..."
3,1309123451433746437,TheLegitIcon2,6458,37979,"Phumlani, 𝒍𝒐𝒗𝒆𝒓, 𝒇𝒖𝒕𝒖𝒓𝒆 𝒍𝒆𝒂𝒅𝒆𝒓 𝒂𝒏𝒅 𝒂 𝒈𝒐𝒐𝒅 𝒄𝒉𝒂𝒓...","Pietermaritzburg, South Africa",At least offer ukumbhalela ama board for that ...,2022-09-29 11:10:06+00:00,0,0,1,0,{'place_id': '52e073e7724385c3'},"Pietermaritzburg, South Africa","[30.2563496, -29.6999989, 30.4512134, -29.5328..."
4,177685467,Nwabisa_1,1878,52381,"mamCirha, Qhanqolo, Ncibane , Nojawolo, Ntswen...",Joburg,We used to catch a train from Ikwezi to Mzimhl...,2022-09-29 10:52:13+00:00,1,0,1,0,{'place_id': '3e46a98adcf05e59'},"Meadowlands, South Africa","[27.8651453, -26.2359005, 27.9219768, -26.1964..."


In [24]:
df['place_name'].unique()

array(['South Africa', 'Benoni, South Africa', 'Pretoria, South Africa',
       'Pietermaritzburg, South Africa', 'Meadowlands, South Africa',
       'Kokosi, South Africa', 'Seshego, South Africa',
       'Randburg, South Africa', 'Germiston, South Africa',
       'Ermelo, South Africa', 'Durban, South Africa',
       'Johannesburg, South Africa', 'Uvongo, South Africa',
       'Phalaborwa, South Africa', 'Boksburg, South Africa',
       'Kempton Park, South Africa', 'Nelspruit, South Africa',
       'Roodepoort, South Africa', 'Aliwal North, South Africa',
       'Hazyview, South Africa', 'Port Elizabeth, South Africa',
       'Mbombela Municipality', 'White River, South Africa',
       'Chatsworth, South Africa', 'Midrand, South Africa',
       'Cape Town, South Africa', 'Stellenbosch, South Africa',
       'Sandton, South Africa', 'Centurion, South Africa',
       'Ibhayi, South Africa', 'Milnerton, South Africa',
       'Rustenburg, South Africa', 'Walkerville, South Africa',
    